In [28]:
%load_ext autoreload
%autoreload 2

from aiida import orm, engine, load_profile
from aiida.common import NotExistent

import aiida_dislocation
import importlib
importlib.reload(aiida_dislocation.workflows.usf)
from aiida_dislocation.workflows.usf import USFWorkChain

from ase.io import read
from ase.build import bulk
from aiida.engine import run

import logging


load_profile()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Profile<uuid='155a05c5c8f7420aa49bbf9207908698' name='supercon'>

In [21]:
prepend_text = """
module load intel
module load mkl
module load impi
module load fftw/3.3.10
"""

group_label = 'test/workchains/USF/'
test_USF_group, created = orm.Group.collection.get_or_create(label=group_label)
if created:
    print(f"Group '{group_label}' created.")
else:
    print(f"Group '{group_label}' already exists.")


computer = 'mn5'
mn5_computer = orm.load_computer(computer)
mn5_computer.set_workdir('/gpfs/scratch/ehpc14/uclo653525/aiida_workdir/test_aiida_dislocation')

version = '730'
pw_label = f'qe-{version}-pw'
pw_code = orm.load_code(f'{pw_label}@{computer}')


Group 'test/workchains/USF/' already exists.


# TEST A1 -- Aluminium

In [12]:
import yaml

ase_atom = read('./structures/cif/Al-225-A1.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5


## {100} surface

In [13]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A1', '100'])

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 10:17:46 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71643|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
04/28/2025 10:17:46 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71643|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
04/28/2025 10:17:46 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71643|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is AB.Slipping direction: None. 
04/28/2025 10:17:47 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71643|USFWorkChain|run_scf]: launching PwCalculation<71644> for Al unit cell geometry.
04/28/2025 10:21:26 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71643|USFWorkChain|inspect_scf]: SCF calculation<7

## {110} surface

In [15]:
inputs['skip_uc'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A1', '110'])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 10:40:12 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71677|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
04/28/2025 10:40:12 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71677|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
04/28/2025 10:40:12 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71677|USFWorkChain|validate_slipping_system]: Faulting is possible for 110 gliding plane.The stacking order is AB. Slipping direction: 1/2[112]. 
04/28/2025 10:40:12 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [WARNING] You are skipping the unitcell calculation. Hopefully you know what you are doing!
04/28/2025 10:40:12 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71677|USFWorkChain|run_USF]: Multiplicity of the f

## {111} surface

In [7]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(9)
inputs['slipping_system'] = orm.List(['A1', '111', None])

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 09:18:10 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|validate_slipping_system]: We are doing FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
04/28/2025 09:18:10 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|validate_slipping_system]: ABC stacking for {111} gliding plane. Faulting possible. slipping direction is along 1/2[110].
04/28/2025 09:18:10 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|run_scf]: launching PwCalculation<71585> for Al unit cell geometry.
04/28/2025 09:21:02 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|inspect_scf]: SCF calculation<71585> finished
04/28/2025 09:21:02 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|run_USF]: Multiplicity of the faulted geometr

# TEST A2 -- Vanadium

In [26]:
import yaml

ase_atom = read('./structures/cif/V-227-A2.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([14, 14, 14])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.01
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5



## {100} surface

In [ ]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(16)
inputs['slipping_system'] = orm.List(['A2', '100'])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 10:44:28 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71705|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #227, prototype V>. I don't know the usual gliding plane. 
04/28/2025 10:44:28 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71705|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
04/28/2025 10:44:28 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71705|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is AB. Slipping direction: 1/2[110]. 
04/28/2025 10:44:28 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71705|USFWorkChain|run_scf]: launching PwCalculation<71706> for V unit cell geometry.
04/28/2025 10:48:08 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71705|USFWorkChain|inspect_scf]: SCF calcul

## {110} surface

In [27]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A2', '110'])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 11:50:29 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71789|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #227, prototype V>. I don't know the usual gliding plane. 
04/28/2025 11:50:29 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71789|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
04/28/2025 11:50:29 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71789|USFWorkChain|validate_slipping_system]: Faulting is possible for 110 gliding plane.The stacking order is AB. Slipping direction: 1/2[001]. 
04/28/2025 11:50:30 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71789|USFWorkChain|run_scf]: launching PwCalculation<71790> for V unit cell geometry.
04/28/2025 11:55:20 AM <158968> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71789|USFWorkChain|inspect_scf]: SCF calcul

# TEST HALF-HEUSLER

In [11]:

import yaml

ase_atom = read('./structures/cif/mpds-S1612207-NbCoSb-216-C1b.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'minimal'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.01
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5



## {100} surface

In [12]:
inputs['skip_uc'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['C1b', '110', None])

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)

04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|validate_slipping_system]: We are doing half-heusler-type structure, space group (216).
04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|validate_slipping_system]: AB stacking for {110} gliding plane. Faulting possible.slipping direction is along 1/2[-111].
04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [WARNING] You are skipping the unitcell calculation.Hopefully you know what you are doing!
04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|run_USF]: Multiplicity of the faulted geometry: uuid: 5bc1e3a8-792c-4a50-94e7-53ed01b2253d (unstored) value: 8.0
04/26/2025 12:11:01 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|run_USF]